In [110]:
import pandas as pd
box_placement = pd.read_csv(r"C:\Users\digit\OneDrive\Documents\Datasets\BoxPlacement.csv")
wet_harvest = pd.read_csv(r"C:\Users\digit\OneDrive\Documents\Datasets\wetharvest.csv")
dry_harvest = pd.read_csv(r"C:\Users\digit\OneDrive\Documents\Datasets\dryharvest.csv")

In [111]:

# Merge datasets based on respective @case_id columns
merged_df = box_placement.merge(wet_harvest, left_on="b.@case_id", right_on="w.@case_id", how="inner")\
    .merge(dry_harvest, left_on="b.@case_id", right_on="d.@case_id", how="inner")

# Drop the redundant @case_id columns
merged_df.drop(columns=["w.@case_id", "d.@case_id"], inplace=True)

# Save the merged dataset if needed
merged_df.to_csv("merged_dataset.csv", index=False)


In [112]:
# Display the first few rows of the DataFrame
print(merged_df.head())

  b.client_name b.distance_to_water_body b.enumerator_comment   
0             A           More_than_5_KM     successfuly done  \
1             A           More_than_5_KM                   ok   
2             A           More_than_5_KM                   ok   
3             A           More_than_5_KM                   ok   
4             A           More_than_5_KM                   ok   

  b.expected_harvest_date b.farmer_in_list_or_control b.farmer_state_district   
0              2022-05-27              control_farmer                 Katsina  \
1              2022-06-08              control_farmer                 Zamfara   
2              2022-06-06              control_farmer                 Zamfara   
3              2022-06-16              control_farmer                 Zamfara   
4              2022-06-13              control_farmer                 Zamfara   

  b.field_irrigated b.insured_crop b.intercropping   
0                no          wheat              no  \
1             

In [113]:
# Assuming merged_df is your DataFrame
# Display all columns without truncation
# Display all columns without truncation
pd.set_option('display.max_columns', None)

print(merged_df.columns)


Index(['b.client_name', 'b.distance_to_water_body', 'b.enumerator_comment',
       'b.expected_harvest_date', 'b.farmer_in_list_or_control',
       'b.farmer_state_district', 'b.field_irrigated', 'b.insured_crop',
       'b.intercropping', 'b.other_factors_that_affected_germination',
       ...
       'was_anything_added_or_removed_from_the_harvest_bag', 'd.timeEnd',
       'd.timeStart', 'cannot_proceed_with_dry_harvest', 'd.username',
       'dry_harvest_comment', 'd.latitude', 'd.longitude', 'd.altitude',
       'd.accuracy'],
      dtype='object', length=121)


In [114]:
'''
2. You are to find enumerators who are likely not entering correct data (use username column to identify enumerators).  Some indications of this are:
               1. Crazy box dimensions
               2. False zero yields
              3. Dry weight exceeding wet weight
              4. Non-compliant data sets i.e no box dimension yet there is yield data captured. Zero wet weight yield but greater than zero dry weight, harvest crop mixed with other crops
              5. Note that other variables in the dataset might also explain funny results, check other variables.  You can also look for those types of variables. 
'''



# Function to identify enumerators with suspicious data
def identify_suspicious_enumerators(merged_df):
    # Initialize a list to store suspicious enumerators
    suspicious_enumerators = []

    # 1. Check for crazy box dimensions
    crazy_box_dimensions = merged_df[(merged_df['box1_length'] > 8) | (merged_df['box1_width'] > 5) | (merged_df['box2_length'] > 8) | (merged_df['box2_width'] > 5)]
    suspicious_enumerators.extend(crazy_box_dimensions['b.username'].unique())
    
    # 2. Check for false zero yields
    false_zero_yields = merged_df[((merged_df['box1_wet_weight'] == 0) & (merged_df['box1_dry_weight'] != 0)) |
                             ((merged_df['box1_dry_weight'] == 0) & (merged_df['box1_wet_weight'] != 0)) |
                                 ((merged_df['box2_wet_weight'] == 0) & (merged_df['box2_dry_weight'] != 0)) |
                                  ((merged_df['box2_dry_weight'] == 0) & (merged_df['box2_wet_weight'] != 0)) |
                                 ((merged_df['box1_wet_weight_confirmation'] == 0) & (merged_df['box1_dry_weight_confirmation'] != 0)) |
                                 ((merged_df['box1_dry_weight_confirmation'] == 0) & (merged_df['box1_wet_weight_confirmation'] != 0))]
    suspicious_enumerators.extend(false_zero_yields['b.username'].unique())
    
        # 3. Check for dry weight exceeding wet weight
    dry_weight_exceeds_wet_weight = merged_df[merged_df['box1_dry_weight'] > merged_df['box1_wet_weight']]
    suspicious_enumerators.extend(dry_weight_exceeds_wet_weight['b.username'].unique())
    

        # 3. Check for dry weight exceeding wet weight
    dry_weight_exceeds_wet_weight2 = merged_df[merged_df['box2_dry_weight'] > merged_df['box2_wet_weight']]
    suspicious_enumerators.extend(dry_weight_exceeds_wet_weight2['b.username'].unique())
    
        # 3. Check for dry weight exceeding wet weight
    dry_weight_exceeds_wet_weight3 = merged_df[merged_df['box1_dry_weight_confirmation'] > merged_df['box1_wet_weight_confirmation']]
    suspicious_enumerators.extend(dry_weight_exceeds_wet_weight3['b.username'].unique())
    
    # 3. Check for dry weight exceeding wet weight
    dry_weight_exceeds_wet_weight4 = merged_df[merged_df['box2_dry_weight_confirmation'] > merged_df['box2_wet_weight_confirmation']]
    suspicious_enumerators.extend(dry_weight_exceeds_wet_weight4['b.username'].unique())
    
    # 4. Check for non-compliant data sets
    non_compliant_data = merged_df[((merged_df['box1_length'].isnull()) & (merged_df['box1_wet_weight'] != 0)) |
                              ((merged_df['box2_length'].isnull()) & (merged_df['box2_wet_weight'] != 0))]
    suspicious_enumerators.extend(non_compliant_data['b.username'].unique())

    
    # 5. Check for other suspicious comments
    suspicious_comments = merged_df[merged_df['dry_harvest_comment'].str.contains('successful|good|Successfuly|Done|satisfied', case=False) & (merged_df['cannot_proceed_with_dry_harvest'] == 'OK')]
    suspicious_enumerators.extend(suspicious_comments['b.username'].unique())
    
    # Return unique list of suspicious enumerators
    return list(set(suspicious_enumerators))

# Identify suspicious enumerators
suspicious_enumerators = identify_suspicious_enumerators(merged_df)
print("Suspicious enumerators:", suspicious_enumerators)

Suspicious enumerators: ['nig120', 'nig118', 'nig097', 'nig102', 'nig036', 'nig021', 'nig090', 'nig108', 'nig033', 'nig103', 'nig099', 'nig035', 'nig075', 'nig175', 'nig186', 'nig034', 'nig133', 'nig106', 'nig031', 'nig180', 'nig137', 'nig030', 'nig109', 'nig105', 'nig183', 'nig039', 'nig174', 'nig188', 'nig027', 'nig028', 'nig098', 'nig187', 'nig044', 'nig025', 'nig100', 'nig111', 'nig048', 'nig185', 'nig184', 'nig104', 'nig114', 'nig026', 'nig037', 'nig168', 'nig132', 'nig024']


<iframe src="https://www.google.com/maps/d/u/0/embed?mid=1mrpeZHBwMf6on0nmzovsjKFchZJTYiI&ehbc=2E312F&noprof=1" width="640" height="480"></iframe>


<iframe src="https://www.google.com/maps/d/u/0/embed?mid=1mrpeZHBwMf6on0nmzovsjKFchZJTYiI&ehbc=2E312F&noprof=1" width="640" height="480"></iframe>


In [115]:
'''
3. Use the box placement latitude and longitude, to come up with a spatial distribution of the data points on a map (you can use tools such as QGIS, Google Maps, ArcGIS or whichever tool you deem fit).  Consider this as well when assessing enumerator performance
'''

from IPython.display import HTML

# HTML code for embedding the map
html_code = '''
<iframe src="https://www.google.com/maps/d/u/0/embed?mid=1mrpeZHBwMf6on0nmzovsjKFchZJTYiI&ehbc=2E312F&noprof=1" width="640" height="480"></iframe>
'''

# Display the HTML content
HTML(html_code)


In [116]:
'''
4. Compute the average yield in Mt/Ha for each observation, using the dry weight
'''

# Fill specific columns with 0 if null
columns_to_fill_with_zero = ['box1_dry_weight', 'box2_dry_weight', 'box1_wet_weight','box2_wet_weight']

merged_df[columns_to_fill_with_zero] = merged_df[columns_to_fill_with_zero].fillna(0)


# Calculate total weight in kg
merged_df['total_weight_kg'] = merged_df['box1_dry_weight'] + merged_df['box2_dry_weight']

# Convert total weight to metric tonnes
merged_df['total_weight_metric_tonnes'] = merged_df['total_weight_kg'] / 1000

# Calculate area of Box 1 and Box 2
merged_df['area_box1'] = merged_df['box1_length'] * merged_df['box1_width']
merged_df['area_box2'] = merged_df['box2_length'] * merged_df['box2_width']

# Calculate total area in square meters
merged_df['total_area_sq_m'] = merged_df['area_box1'] + merged_df['area_box2']

# Convert total area to hectares
merged_df['total_area_hectares'] = merged_df['total_area_sq_m'] / 10000

# Calculate average yield in Mt/Ha
merged_df['average_yield_Mt_Ha'] = (merged_df['total_weight_metric_tonnes'] / merged_df['total_area_hectares']).round(2)

# Display the result
print(merged_df['average_yield_Mt_Ha'])


0       1.17
1       0.68
2       0.59
3       0.79
4       0.58
        ... 
2182    1.48
2183    1.11
2184    1.67
2185    1.13
2186    1.48
Name: average_yield_Mt_Ha, Length: 2187, dtype: float64


In [117]:
'''
5. Create filters in a new column or columns to determine which types of yields we should consider outliers or errors
and fill those columns so that we can filter the outliers on and off as needed. (if using code, clearly show these filters)
'''
import pandas as pd
import numpy as np

# Assuming merged_df is your DataFrame
# First, create DataFrames for each condition
condition1 = merged_df['box1_dry_weight'] <= merged_df['box1_wet_weight']
condition2 = merged_df['box2_dry_weight'] <= merged_df['box2_wet_weight']

# Then, concatenate them along the columns axis
conditions_df = pd.concat([condition1, condition2], axis=1)

# Rename the columns
conditions_df.columns = ['Dry_Custom', 'Dry_Custom2']

# Now, you can assign this DataFrame to merged_df
merged_df = pd.concat([merged_df, conditions_df], axis=1)

# When getting the yield we will only account for average yield where Dry_Custom and Dry_Custom2 column is TRUE.


In [118]:
'''
6. Identify major problems affecting the crops per district or state. Use box1_problem column. Is there consistency?
'''
import pandas as pd

# Grouping by state or district and aggregating problems
problem_counts = merged_df.groupby(['b.farmer_state_district', 'box1_problem']).size().reset_index(name='count')

# Displaying the results
print(problem_counts)


   b.farmer_state_district                box1_problem  count
0                  Adamawa  animal_cattle_encroachment      1
1                  Adamawa                     drought     12
2                  Adamawa                       flood      1
3                  Adamawa               late_planting     35
4                  Adamawa          locust_infestation      5
..                     ...                         ...    ...
58                    Yobe               late_planting     36
59                    Yobe            poor_germination     13
60                    Yobe                       weeds      1
61                 Zamfara                     drought     44
62                 Zamfara            poor_germination      3

[63 rows x 3 columns]


In [120]:

import pandas as pd

# Grouping by state or district and aggregating problems
problem_counts = merged_df.groupby(['b.farmer_state_district', 'box1_problem']).size().reset_index(name='count')

# Getting the maximum count for each 'b.farmer_state_district'
max_counts = problem_counts.groupby('b.farmer_state_district')['count'].max()

# Filtering the DataFrame to show the corresponding 'box1_problem'
max_counts_df = problem_counts.merge(max_counts, how='inner', on=['b.farmer_state_district', 'count'])

# Displaying the results
print(max_counts_df)


   b.farmer_state_district      box1_problem  count
0                  Adamawa  poor_germination     40
1                   Bauchi  poor_germination     78
2                    Borno           drought     33
3                   Jigawa     late_planting      5
4                   Kaduna  poor_germination     47
5                     Kano  poor_germination     16
6                  Katsina  poor_germination     25
7                    Niger     late_planting    153
8                   Sokoto     late_planting     18
9                   Taraba  poor_germination     66
10                    Yobe     late_planting     36
11                 Zamfara           drought     44


In [121]:
# Save the merged dataset if needed
merged_df.to_csv("merged_df.csv", index=False)

In [106]:
def identify_suspicious_enumerators(merged_df):
    # Initialize a list to store suspicious enumerators
    suspicious_enumerators = []

    # 1. Check for crazy box dimensions
    crazy_box_dimensions = merged_df[(merged_df['box1_length'] > 8) | (merged_df['box1_width'] > 5) | (merged_df['box2_length'] > 8) | (merged_df['box2_width'] > 5)]
    suspicious_enumerators.extend(crazy_box_dimensions['b.username'].unique())
    
    # 2. Check for false zero yields
    false_zero_yields = merged_df[((merged_df['box1_wet_weight'] == 0) & (merged_df['box1_dry_weight'] != 0)) |
                             ((merged_df['box1_dry_weight'] == 0) & (merged_df['box1_wet_weight'] != 0)) |
                                 ((merged_df['box2_wet_weight'] == 0) & (merged_df['box2_dry_weight'] != 0)) |
                                  ((merged_df['box2_dry_weight'] == 0) & (merged_df['box2_wet_weight'] != 0)) |
                                 ((merged_df['box1_wet_weight_confirmation'] == 0) & (merged_df['box1_dry_weight_confirmation'] != 0)) |
                                 ((merged_df['box1_dry_weight_confirmation'] == 0) & (merged_df['box1_wet_weight_confirmation'] != 0))]
    suspicious_enumerators.extend(false_zero_yields['b.username'].unique())
    
    
    
    # Return unique list of suspicious enumerators
    return list(set(suspicious_enumerators))

# Identify suspicious enumerators
suspicious_enumerators = identify_suspicious_enumerators(merged_df)
print("Suspicious enumerators:", suspicious_enumerators)

Suspicious enumerators: ['nig187', 'nig137', 'nig174']


In [107]:
def identify_suspicious_enumerators(merged_df):
    # Initialize a list to store suspicious enumerators
    suspicious_enumerators = []   
    
    # 3. Check for dry weight exceeding wet weight
    dry_weight_exceeds_wet_weight = merged_df[merged_df['box1_dry_weight'] > merged_df['box1_wet_weight']]
    suspicious_enumerators.extend(dry_weight_exceeds_wet_weight['b.username'].unique())
    
        # Return unique list of suspicious enumerators
    return list(set(suspicious_enumerators))

# Identify suspicious enumerators
suspicious_enumerators = identify_suspicious_enumerators(merged_df)
print("Suspicious enumerators:", suspicious_enumerators)

Suspicious enumerators: ['nig120', 'nig097', 'nig102', 'nig090', 'nig108', 'nig033', 'nig103', 'nig099', 'nig075', 'nig034', 'nig106', 'nig031', 'nig030', 'nig105', 'nig039', 'nig174', 'nig188', 'nig028', 'nig098', 'nig187', 'nig044', 'nig100', 'nig111', 'nig048', 'nig185', 'nig104', 'nig026', 'nig037', 'nig168', 'nig132']


In [108]:
def identify_suspicious_enumerators(merged_df):
    # Initialize a list to store suspicious enumerators
    suspicious_enumerators = []   
    
        # 4. Check for non-compliant data sets
    non_compliant_data = merged_df[((merged_df['box1_length'].isnull()) & (merged_df['box1_wet_weight'] != 0)) |
                              ((merged_df['box2_length'].isnull()) & (merged_df['box2_wet_weight'] != 0))]
    suspicious_enumerators.extend(non_compliant_data['b.username'].unique())
    
    
        
        # Return unique list of suspicious enumerators
    return list(set(suspicious_enumerators))

# Identify suspicious enumerators
suspicious_enumerators = identify_suspicious_enumerators(merged_df)
print("Suspicious enumerators:", suspicious_enumerators)

Suspicious enumerators: []


In [109]:
def identify_suspicious_enumerators(merged_df):
    # Initialize a list to store suspicious enumerators
    suspicious_enumerators = []   
    
        # 5. Check for other suspicious comments
    suspicious_comments = merged_df[merged_df['dry_harvest_comment'].str.contains('successful|good|Successfuly|Done|satisfied', case=False) & (merged_df['cannot_proceed_with_dry_harvest'] == 'OK')]
    suspicious_enumerators.extend(suspicious_comments['b.username'].unique())
    
            
        # Return unique list of suspicious enumerators
    return list(set(suspicious_enumerators))

# Identify suspicious enumerators
suspicious_enumerators = identify_suspicious_enumerators(merged_df)
print("Suspicious enumerators:", suspicious_enumerators)

Suspicious enumerators: ['nig109', 'nig114', 'nig188', 'nig186', 'nig137']


In [ ]:
#nig137 is present in the all the lists above of Suspicious enumerators